In [1]:
import pandas as pd
from datetime import datetime as dt
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is /Users/drewrichard/Documents/1 projects/nss/nss_projects/accre-carbonara/notebooks


In [3]:
jobs = pd.read_csv("../data/fullsample.csv",
                    # nrows = 100000
                    )
ce5 = pd.read_csv('../data/ce5_unresponsive.csv')
ce6 = pd.read_csv('../data/ce6_unresponsive.csv')
jobs = jobs[jobs['END'] != 'Unknown']
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])
jobs['DIFFMEM'] = jobs['REQMEM'] - jobs['USEDMEM']
jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)
jobs.head(5)


,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,DIFFMEM
1,30853133,COMPLETED,1,2021-08-06 11:36:09,2021-09-05 11:36:32,262144,Mn,20604.62,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0,241539.38
2,30858137,COMPLETED,1,2021-08-06 19:04:39,2021-09-05 19:04:53,204800,Mn,57553.77,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0,147246.23
3,30935078,COMPLETED,1,2021-08-09 16:52:51,2021-09-07 20:52:55,65536,Mn,20577.96,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0,44958.04
4,31364111_2,COMPLETED,1,2021-08-17 07:45:07,2021-09-10 16:45:24,16384,Mn,9733.43,24-09:00:00,24-09:00:17,1,1,production,0:0,6650.57
5,31364111_3,COMPLETED,1,2021-08-17 07:45:07,2021-09-06 16:17:34,16384,Mn,9708.04,24-09:00:00,20-08:32:27,1,1,production,0:0,6675.96


In [4]:
len(jobs)


7395234

In [ ]:
jobs_ce56 = pd.concat([jobs, ce5, ce6])
jobs_ce56['END'] = pd.to_datetime(jobs_ce56['END']).dt.floor('s')
jobs_ce56.sort_values('END')


,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,DIFFMEM,USER,RETRY,TIME,RETURNCODE,COMMAND
6640577,24460577,COMPLETED,1.0,2020-10-01 00:05:55,2020-10-01 00:10:15,4096.0,Mc,868.77,12:00:00,00:04:20,1.0,1.0,production,0:0,3227.23,NaN,NaN,NaN,NaN,NaN
6640635,24460647,COMPLETED,1.0,2020-10-01 00:10:38,2020-10-01 00:12:58,2000.0,Mn,0.09,2-00:00:00,00:02:20,1.0,1.0,sam,0:0,1999.91,NaN,NaN,NaN,NaN,NaN
6640627,24460639,COMPLETED,1.0,2020-10-01 00:11:22,2020-10-01 00:13:31,1024.0,Mc,221.10,00:30:00,00:02:09,1.0,1.0,production,0:0,802.90,NaN,NaN,NaN,NaN,NaN
6640651,24460668,COMPLETED,1.0,2020-10-01 00:15:16,2020-10-01 00:17:23,2000.0,Mn,0.09,2-00:00:00,00:02:07,1.0,1.0,sam,0:0,1999.91,NaN,NaN,NaN,NaN,NaN
6640654,24460671,COMPLETED,1.0,2020-10-01 00:17:08,2020-10-01 00:17:53,1024.0,Mc,636.96,00:30:00,00:00:45,1.0,1.0,production,0:0,387.04,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927386,32923987,COMPLETED,1.0,2021-10-07 20:09:26,2021-10-07 20:11:21,2000.0,Mn,0.09,2-00:00:00,00:01:55,1.0,1.0,sam,0:0,1999.91,NaN,NaN,NaN,NaN,NaN
927387,32923988,COMPLETED,1.0,2021-10-07 20:15:05,2021-10-07 20:17:24,2000.0,Mn,0.09,2-00:00:00,00:02:19,1.0,1.0,sam,0:0,1999.91,NaN,NaN,NaN,NaN,NaN
927388,32923989,COMPLETED,1.0,2021-10-07 20:24:26,2021-10-07 20:26:01,2000.0,Mn,393.07,2-00:00:00,00:01:35,1.0,1.0,sam,0:0,1606.93,NaN,NaN,NaN,NaN,NaN
1491977,32923996,COMPLETED,1.0,2021-10-07 20:31:08,2021-10-07 20:32:45,2000.0,Mn,0.09,2-00:00:00,00:01:37,1.0,1.0,sam,0:0,1999.91,NaN,NaN,NaN,NaN,NaN
